In [1]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

import esri_credentials

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from IPython.display import display
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection


#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"
#output_layer_name = 'ff84ab8579a240529b7db45a569b71ff'
output_layer_name = "8f990d7f36404f4f8cb54c0cb229ab5f"
max_record_count = 250_000

df = pd.read_pickle("./cleaned_df.p")

lahub_user = esri_credentials.preston_user
lahub_pass = esri_credentials.preston_pw

In [3]:
gis = GIS('https://lahub.maps.arcgis.com',  username=lahub_user, password=lahub_pass)

flayer = gis.content.get(output_layer_name)
ActiveBusinesses_flayer = flayer.layers[0].query()
flayer_collection = FeatureLayerCollection.fromitem(flayer)

existing_table = ActiveBusinesses_flayer.sdf

In [4]:
updated_table = pd.merge(
    existing_table[["OBJECTID", "GEOID10", "SHAPE"]],
    # Rename columns to match, drop columns not in layer
    # ESRI renamed the geometry to be SHAPE...and it's stored slightly different than in geodataframe
    (df.drop(columns = ["geometry"])
    ), 
    on = "GEOID10", 
    how = "inner")

updated_table.to_csv("./upload_me.csv", index=False)

In [5]:
# Upload new item to begin with
#item_prop = {"title": "ActiveBizTest"}
#gis.content.add(item_properties=item_prop, data="./upload_me.csv")

In [7]:
# Overwrite table
flayer_collection.manager.overwrite("./upload_me.csv")

Exception: User does not have permissions to access this service
User does not have permissions to access this service
(Error Code: 403)

In [68]:
activebiz_flayer = flayer.layers[0]
layer_fset = activebiz_flayer.query()
features_for_update = [] #list containing corrected features
all_features = layer_fset.features

In [69]:
layer_rename_columns_dict = {
    'Accommodation and Food Services': 'Accommodation_and_Food_Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Administrative_and_Support_and_',
    'Agriculture, Forestry, Fishing and Hunting': 'Agriculture__Forestry__Fishing_',
    'Arts, Entertainment, and Recreation': 'Arts__Entertainment__and_Recrea',
    'Construction': 'Construction',
    'Educational Services': 'Educational_Services',
    'Finance and Insurance': 'Finance_and_Insurance',
    'Health Care and Social Assistance': 'Health_Care_and_Social_Assistan',
    'Information': 'Information',
    'Manufacturing': 'Manufacturing',
    'Medical Marijuana Collective': 'Medical_Marijuana_Collective',
    'Mining': 'Mining',
    'Not Classified': 'Not_Classified',
    'Other Services (except Public Administration)': 'Other_Services__except_Public_A',
    'Professional, Scientific, and Technical Services': 'Professional__Scientific__and_T',
    'Real Estate Rental and Leasing': 'Real_Estate_Rental_and_Leasing',
    'Retail Trade': 'Retail_Trade',
    'Transportation and Warehousing': 'Transportation_and_Warehousing',
    'Utilities': 'Utilities',
    'Wholesale Trade': 'Wholesale_Trade'                                               
}

In [78]:
wrong_geoid = ["060375722021"]
overlap_rows[overlap_rows.GEOID10.isin(wrong_geoid)]

,GEOID10,CTBG10,CT10,AreaSqMil,LABEL,FIP10,FIP10RV,CDP_NAME,CITYNAME,COMMNAME,...,Medical_Marijuana_Collective,Mining,Not_Classified,Other_Services__except_Public_A,Professional__Scientific__and_T,Real_Estate_Rental_and_Leasing,Retail_Trade,Transportation_and_Warehousing,Utilities,Wholesale_Trade
143,060375722021,5722021,572202,0.2191,BG 1,43000,43000,Long Beach,Long Beach,City of Long Beach,...,0,0,0,0,0,0,0,0,0,0


In [77]:
df[df.GEOID10.isin(wrong_geoid)]

,GEOID10,CTBG10,CT10,AreaSqMil,LABEL,FIP10,FIP10RV,CDP_NAME,CITYNAME,COMMNAME,...,Medical_Marijuana_Collective,Mining,Not_Classified,Other_Services__except_Public_A,Professional__Scientific__and_T,Real_Estate_Rental_and_Leasing,Retail_Trade,Transportation_and_Warehousing,Utilities,Wholesale_Trade
143,060375722021,5722021,572202,0.2191,BG 1,43000,43000,Long Beach,Long Beach,City of Long Beach,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
from copy import deepcopy

overlap_rows = df[df.GEOID10.isin(layer_fset.sdf.GEOID10)]
overlap_rows.to_csv("./upload_csv", index=False)

csv_table = pd.read_csv("./upload_me.csv", dtype = {"GEOID10": "str"})

for GEOID10 in overlap_rows['GEOID10']:
    print(GEOID10)
    # get the feature to be updated
    original_feature = [f for f in all_features if f.attributes['GEOID10'] == GEOID10][0]
    feature_to_be_updated = deepcopy(original_feature)

    # get the matching row from csv
    matching_row = csv_table.where(overlap_rows.GEOID10 == GEOID10).dropna()
    print("Matching Row")
    display(matching_row)
    # assign the updated values
    
    for col in layer_rename_columns_dict.values():
        try:
            feature_to_be_updated.attributes[col] = matching_row[col].values[0]
        except:
            print(GEOID10)
            print(col)
            display(feature_to_be_updated.attributes[col])
            display(matching_row[col])

In [ ]:
flayer_collection.manager.update_definition({"maxRecordCount": max_record_count})